In [138]:
import pandas as pd
pd.set_option("display.max_row", 1000)
pd.set_option("display.max_column", 100)
import warnings
warnings.filterwarnings('ignore')

path1 = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"
path2 = "https://raw.githubusercontent.com/Soyoung-Yoon/data_01/main/"

8-1) 레스토랑의 팁(tip)분석

다음 조건을 만족하는 분석을 수행하시오.

- 요일(day)별 total_bill의 평균이 전체 total_bill 평균보다 높은 요일의 데이터를 추출하시오.
- 위에서 추출된 데이터를 사용하여, day와 smoker로 그룹화하여 tip의 평균을 구하고, 가장 평균 tip이 높은 (day, smoker) 조합을 찾으시오.
- 위의 가장 평균 tip이 높은 조합의 데이터에 대해, total_bill의 표준편차를 구하시오.
- 표준편차 값을 반올림하여 소수점 아래 3 자리까지 출력하시오.

In [ ]:
df = pd.read_csv(path1 + "tips.csv")
# print(df.head(3))

# 요일(day)별 total_bill의 평균이 전체 total_bill 평균보다 높은 요일의 데이터를 추출
df2 = df.groupby('day')['total_bill'].mean().reset_index()
# print(df2)
days = df2[df2['total_bill'] > df['total_bill'].mean()]['day']
# print(days) # Sat, Sun
df3 = df.loc[df['day'].str.contains('Sat|Sun')].copy()
# print(df3.head())

# 위에서 추출된 데이터를 사용하여, day와 smoker로 그룹화하여 tip의 평균을 구하고, 
# 가장 평균 tip이 높은 (day, smoker) 조합을 찾으시오.
df4 = df3.groupby(['day','smoker'])[['tip']].mean().reset_index().sort_values(by='tip', ascending=False).values[3]
# print(df4[0], df4[1]) # Sat, Yes

# 위의 가장 평균 tip이 높은 조합의 데이터에 대해, total_bill의 표준편차를 구하시오.
# 표준편차 값을 반올림하여 소수점 아래 3 자리까지 출력
cond = (df['day'] == 'Sat') & (df['smoker'] == 'Yes')
result = round(df[cond]['total_bill'].std(), 3)
# print(result) # 10.069


10.069


**8-2a) 프로모션 기획**

온라인 플랫폼에서는 특정 요일에 이벤트의 발생 빈도를 활용한 프로모션을 기획하려 한다.

다음 절차에 따라 문제를 해결하시오.
- 시계열 데이터를 이용해 각 이벤트가 발생한 '요일'을 파생 변수로 추가한 뒤, 요일별 value 총합을 구하시오.
- 위의 데이터를 사용하여, 가장 큰 3개 값의 평균을 구해 반올림하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head())

# 시계열 데이터를 이용해 각 이벤트가 발생한 '요일'을 파생 변수로 추가한 뒤, 요일별 value 총합을 구하시오.
df['date'] = pd.to_datetime(df['date'])
df['요일'] = df['date'].dt.day_name('ko_KR')
# print(df.head(3))
df2 = df.groupby('요일')['value'].sum().reset_index(name='총합').sort_values(by='총합', ascending=False)
# print(df2)

# 위의 데이터를 사용하여, 가장 큰 3개 값의 평균을 구해 반올림하여 정수로 출력하시오.
result = round(df2['총합'][:3].mean())
# print(result) # 8342


8342


**8-2b) 프로모션 기획**

온라인 플랫폼에서는 특정 요일에 이벤트의 발생 빈도를 활용한 프로모션을 기획하려 한다.

다음 절차에 따라 문제를 해결하시오.
- 각 이벤트가 발생한 날짜에서 '요일' 정보를 추출한다.
- 요일별로 value의 총합을 구하여, 가장 높은 합계를 가진 요일 3개를 식별한다.
- 이 3개 요일에 해당하는 모든 이벤트의 value 평균을 계산하고, 그 값을 반올림하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head(3))

# 각 이벤트가 발생한 날짜에서 '요일' 정보를 추출한다.
df['date'] = pd.to_datetime(df['date'])
df['요일'] = df['date'].dt.day_name('ko_KR')
# print(df.head(3))

# 요일별로 value의 총합을 구하여, 가장 높은 합계를 가진 요일 3개를 식별한다.
df2 = df.groupby('요일')['value'].sum().reset_index(name='총합').sort_values(by='총합', ascending=False).head(3)
days = list(df2['요일'])
# print(days) # ['토요일', '일요일', '수요일']

# 이 3개 요일에 해당하는 모든 이벤트의 value 평균을 계산하고, 그 값을 반올림하여 정수로 출력하시오.
result = round(df.loc[df['요일'].isin(days), 'value'].mean())
# print(result) # 57

57


**8-3) 이벤트별 월간 성과**

다양한 이벤트 카테고리별 월간 성과 분석을 위한 과제이다.

다음 절차에 따라 문제를 해결하시오.
- **주어진 데이터**에 대해 카테고리(category2), 연도(year), 월(month) 단위로 value의 총합을 집계하시오.
- 이후, (category2, year)별로 집계된 값 중 가장 value 합계가 높은 월을 선택하시오.
- 이렇게 선택된 월별 최댓값들을 모두 더한 뒤, 그 합계를 반올림하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_05.csv")
# print(df.head(3))

# 주어진 데이터에 대해 카테고리(category2), 연도(year), 월(month) 단위로 value의 총합을 집계하시오.
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df2 = df.groupby(['category2','month'])['value'].sum().reset_index(name='총합').sort_values(by='총합', ascending=False)
df3 = df.groupby(['year','month'])['value'].sum().reset_index(name='총합').sort_values(by='총합', ascending=False)

# 이후, (category2, year)별로 집계된 값 중 가장 value 합계가 높은 월을 선택하시오.
cat_month = df2[df2['총합'] == df2['총합'].max()]['month'].values[0]
yr_month = df3[df3['총합'] == df3['총합'].max()]['month'].values[0]
# print(cat_month, yr_month)

# 이렇게 선택된 월별 최댓값들을 모두 더한 뒤, 그 합계를 반올림하여 정수로 출력하시오.
value1 = df2[df2['month'] == cat_month]['총합'].values[0]
value2 = df3[df3['month'] == yr_month]['총합'].values[0]
result = round(value1 + value2)
# print(result) # 3269

3269


**8-4) 이상 감지 및 성능 분석**

시스템 비활성 이벤트의 이상 감지 및 성능 분석을 위한 과제이다.

다음 절차에 따라 문제를 해결하시오.
- 2020년 1월부터 6월까지의 데이터 중 event가 'Idle'인 이벤트만 필터링하시오.
- 월(month)별로 해당 이벤트들의 value 중앙값을 구하시오.
- 해당 월의 value 중앙값을 기준으로 그 달에 발생한 이벤트 중 value가 중앙값 이하인 이벤트만 선택하시오.
- 이렇게 선택된 이벤트들의 총 개수를 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head(3))

# 2020년 1월부터 6월까지의 데이터 중 event가 'Idle'인 이벤트만 필터링하시오.
df['date'] = pd.to_datetime(df['date'])
df2 = df.loc[(df['date'].dt.month >= 1) & (df['date'].dt.month <= 6)]
df3 = df2[df2['event'] == 'Idle']

# 월(month)별로 해당 이벤트들의 value 중앙값을 구하시오.
df3['month'] = df3['date'].dt.month
month_median = df3.groupby('month')['value'].median().reset_index(name='median')
# print(month_median)

# 해당 월의 value 중앙값을 기준으로 그 달에 발생한 이벤트 중 value가 중앙값 이하인 이벤트만 선택하시오.
df['month'] = df['date'].dt.month
df = pd.merge(df, month_median, "left", "month").dropna()
event_list = list(df[df['value'] <= df['median']]['event'])
# print(event_list)

# 이렇게 선택된 이벤트들의 총 개수를 정수로 출력하시오.
result = len(event_list)
# print(result) # 237

237


**8-5) 야간 시간대 이벤트**

다음은 야간 시간대에 발생한 이벤트 유형 분석을 통해 사용자 활동 패턴을 파악하고자 하는 과제이다.

다음 절차에 따라 문제를 해결하시오.
- 먼저, 시간(hour) 정보가 22시부터 02시까지(22:00:00 ~ 02:59:59)인 데이터를 야간 이벤트로 간주한다.
- 이 야간 이벤트를 기준으로, (month, event)별 value 평균값을 구하시오.
- 각 월(month)에서 value 평균이 가장 높은 이벤트 유형을 하나씩 선택하고, 그 중 'Start'와 'Stop'이벤트의 빈도 수 합을 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head())

# 먼저, 시간(hour) 정보가 22시부터 02시까지(22:00:00 ~ 02:59:59)인 데이터를 야간 이벤트로 간주한다.
df['date_time'] = df['date'] + ' ' + df['time']
df['date_time'] = pd.to_datetime(df['date_time'])
# print(df.head(3))
df2 = df.loc[(df['date_time'].dt.hour >= 22) | (df['date_time'].dt.hour >= 0) & (df['date_time'].dt.hour <= 2)]

# 이 야간 이벤트를 기준으로, (month, event)별 value 평균값을 구하시오.
df2['month'] = df2['date_time'].dt.month
df3 = df2.groupby(['month','event'])[['value']].mean().reset_index()
print(df3)

# 각 월(month)에서 value 평균이 가장 높은 이벤트 유형을 하나씩 선택하고, 그 중 'Start'와 'Stop'이벤트의 빈도 수 합을 정수로 출력하시오.


    month    event      value
0       1    Alert  47.531429
1       1     Idle  62.050000
2       1  Measure  85.070000
3       1      Run  53.680000
4       1    Start  17.820000
5       1     Stop  55.224000
6       2    Alert  54.345000
7       2  Measure  32.280000
8       2      Run  45.423333
9       2    Start  64.120000
10      2     Stop  43.770000
11      3    Alert  39.890000
12      3     Idle  50.933333
13      3  Measure  44.430000
14      3      Run  72.098333
15      3    Start  81.150000
16      4    Alert  38.082500
17      4     Idle  49.180000
18      4  Measure  51.423333
19      4      Run  62.550000
20      4    Start  53.303333
21      4     Stop  71.150000
22      5    Alert  32.540000
23      5     Idle  23.970000
24      5  Measure  53.508000
25      5      Run  57.635000
26      5    Start  46.790000
27      5     Stop  82.380000
28      6    Alert  47.802000
29      6     Idle  57.780000
30      6  Measure  54.756667
31      6      Run  55.580000
32      6 

KeyError: 'mean'